<a href="https://colab.research.google.com/github/mhuertascompany/DL_ED127_2021/blob/main/hand-on/day2/TFP_mass_estimate_ED127.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 Matthew Ho, Francois Lanusse.

Licensed under the Apache License, Version 2.0 (the "License");


# Estimating Galaxy Cluster Masses with TensorFlow and TensorFlow Probability

This notebook is bases on an example by: 
  - [@maho3](https://github.com/maho3) (Matt Ho)
  - [@EiffL](https://github.com/EiffL) (Francois Lanusse)

### Overview

In this tutorial, we learn how to combine Keras and TensorFlow Probability to estimate the masses of galaxy clusters using velocity dispersion measurements. See for instance [Ho et al. (2019)](https://arxiv.org/abs/1902.05950) for a state of the art machine learning approach to this problem.

The context for this problem is to be able to build robust mass estimates for 
galaxy clusters, simply by using information from their galaxy members. And in 
particular line of sight velocity information which can be obtained by spectroscopy.




In [ ]:
#@title Imports and Utility functions { display-mode: "form" }
%pylab inline

import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_probability as tfp
tfd = tfp.distributions

# Activate TF2 behavior:
from tensorflow.python import tf2
if not tf2.enabled():
  import tensorflow.compat.v2 as tf
  tf.enable_v2_behavior()
  assert tf2.enabled()

def binned_plot(X, Y, n=10, percentiles=[35, 50], ax=None, **kwargs):
    # Calculation
    calc_percent = []
    for p in percentiles:
        if p < 50:
            calc_percent.append(50-p)
            calc_percent.append(50+p)
        elif p == 50:
            calc_percent.append(50)
        else:
            raise Exception('Percentile > 50')

    bin_edges = np.linspace(X.min()*0.9999, X.max()*1.0001, n+1)

    dtype = [(str(i), 'f') for i in calc_percent]
    bin_data = np.zeros(shape=(n,), dtype=dtype)

    for i in range(n):
        y = Y[(X >= bin_edges[i]) & (X < bin_edges[i+1])]

        if len(y) == 0:
            continue

        y_p = np.percentile(y, calc_percent)

        bin_data[i] = tuple(y_p)

    # Plotting
    if ax is None:
        f, ax = plt.subplots()

    bin_centers = [np.mean(bin_edges[i:i+2]) for i in range(n)]
    for p in percentiles:
        if p == 50:
            ax.plot(bin_centers, bin_data[str(p)], **kwargs)
        else:
            ax.fill_between(bin_centers,
                            bin_data[str(50-p)],
                            bin_data[str(50+p)],
                            alpha=0.2,
                            **kwargs)

    return bin_data, bin_edges


In [ ]:
#Checking for GPU access
if tf.test.gpu_device_name() != '/device:GPU:0':
  print('WARNING: GPU device not found.')
else:
  print('SUCCESS: Found GPU: {}'.format(tf.test.gpu_device_name()))

## Understanding the data

In [ ]:
#@title Loading dataset.
#from google.colab import auth
from astropy.table import Table

#auth.authenticate_user()
bucket_name='ahw2019'

!gsutil cp gs://{bucket_name}/halo_mass_regression/'Rockstar_UM_z=0.117_contam_summary.fits' contam_summary.fits

print('Download complete')

data = Table.read('contam_summary.fits')

In [ ]:
# Let's check the mass distribution in this sample
hist(log10(data['M200c']),100)
xlabel(r'$\log[M_{200c}\ (h^{-1}M_\odot)]$')

In [ ]:
hist2d(log10(data['M200c']), log10(data['Ngal']),64,cmap='gist_stern');
plt.colorbar()
xlabel(r'$\log_{10}[M_{200c}\ (h^{-1}M_\odot)]$')
ylabel(r'$\log_{10}[N_\mathrm{gal}]$')

This plot shows the scaling relation between mass and number of cluster members

In [ ]:
hist2d(log10(data['M200c']), log10(data['sigv']),64,cmap='gist_stern');
plt.colorbar()
xlabel(r'$\log_{10}[M_{200c}\ (h^{-1}M_\odot)]$')
ylabel(r'$\log_{10}[\sigma_v\ (km\ s^{-1})]$')

This plot illustrates the scaling relation between mass and the velocity dispersion of cluster members.

So, we are going to build a regression model, just as the toy examples we just saw, to estimate cluster masses. Instead of doing a regression with only one input parameter, we are going to use 14 input parameters: number of galaxies in the cluster, line of sight veolcity dispersion + some other moments describing the radial distribution of galaxies in the clusters.

In [ ]:
# This cell performs some pre-processing of the input and output features - just run and ignore for now
# Preprocessing some features
data['LogNgal'] = np.log10(data['Ngal'])
data['Logsigv'] = np.log10(data['sigv'])
data['logmass'] = np.log10(data['M200c'])

# Preparing data set
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Split into train and test
inds_random = permutation(len(data))
data_train = data[inds_random[:200000]]
data_test = data[inds_random[200000:]]

# Features to use for regression
X = data[['LogNgal', # log richness
          'Logsigv', # log velocity dispersion
          'R_mean', 'R_std', 'R_skew', 'R_kurt', # descriptive features of cluster member projected radius distribution
          'm_mean', 'm_std', 'm_skew', 'm_kurt', # " of member stellar mass distribution
          'v_mean', 'v_std', 'v_skew', 'v_kurt'  # " of member LOS velocity distribution
         ]].to_pandas().values

X_train = X[inds_random[:200000]]
X_test = X[inds_random[200000:]]

scaler = MinMaxScaler().fit(data['logmass'].reshape((-1,1)))
feature_scaler = StandardScaler().fit(X_train)

X_train = feature_scaler.transform(X_train)
Y_train = np.clip(scaler.transform(data_train['logmass'].reshape((-1,1))), 1e-5,1-1e-5)

X_test = feature_scaler.transform(X_test)
Y_test = np.clip(scaler.transform(data_test['logmass'].reshape((-1,1))),1e-5,1-1e-5)

Y_true = data_test['logmass']

## First approach: Regression

We begin by implementing a simple regression model using a mean squared error loss. This is the standard approach taken by most ML papers on this problem.


In [ ]:
regression_model = keras.Sequential([
    keras.layers.Dense('FILL THIS OUT', activation='relu', input_shape=(14,)),  ## this is your input model
    ## FILL OUT THE NEURAL NETWORK HERE
    keras.layers.Dense(units=1) ## this is your output - WHAT IS THE ACTIVATION FUNCTION HERE?
])

pathout='models/mass1'

#define callbacks

callback = tf.keras.callbacks.LearningRateScheduler(lambda e: 0.001 if e < 2 else 0.0001)  # this is to reduce the LR during training

regression_model.compile(optimizer='adam', metrics=[], "WHAT IS MISSING HERE TO PERFORM A REGRESSION?")

In [ ]:
regression_model.summary()

In [ ]:
history = regression_model.fit("FILL THIS OUT", epochs=5, batch_size=100, callbacks=[callback])

In [ ]:
plot(history.history['loss'])

plt.xlabel('Epochs')
plt.ylabel('Loss')

Now that the model is trained, let's extract some predictions:

In [ ]:
Y_pred = scaler.inverse_transform(regression_model.predict(X_test)).squeeze()

In [ ]:
## EXPLORE THE RESULTS HERE:

f = plt.figure(figsize=(6,8))
gs = mpl.gridspec.GridSpec(2,1,height_ratios=[3,1], hspace=0)

ax1 = f.add_subplot(gs[0,0])

ax1.plot(np.arange(13,16,0.1),np.arange(13,16,0.1),'k--')
_ = binned_plot(Y_true, 
                Y_pred, 
                n=20, percentiles=[35,45,50], 
                color='b', ax=ax1)

ax1.set_xlim(13.5,15.3)
ax1.set_ylim(13.5,15.3)
ax1.set_xticks([])
ax1.set_yticks(ax1.get_yticks()[1:])
ax1.set_ylabel(r'$\log_{10}\left[M_\mathrm{pred}\ (M_\odot h^{-1})\right]$', fontsize=14)

ax2 = f.add_subplot(gs[1,0])

ax2.plot(np.arange(13,16,0.1),[0]*30,'k--')
_ = binned_plot(Y_true, 
                Y_pred - Y_true, 
                n=20, percentiles=[35,45,50], 
                color='b', ax=ax2)
ax2.set_xlim(13.5,15.3)
ax2.set_ylim(-0.5,0.5)
ax2.set_xlabel(r'$\log_{10}\left[M_\mathrm{true}\ (M_\odot h^{-1})\right]$', fontsize=14)
ax2.set_ylabel(r'$\epsilon$', fontsize=14)

We see a significant bias... What could be the cause? What did we do wrong?

## Second approach: Probabilistic Modelling

By now we now that using a MSE can lead to biased estimates. How about we switch to a model of the full posterior, that should fix our problems!


In [ ]:
num_components = 16
event_shape = [1]

params_size = tfp.layers.MixtureSameFamily.params_size(
    num_components,
    component_params_size=tfp.layers.IndependentNormal.params_size(event_shape))

model = keras.Sequential([
    keras.layers.Dense(units=128, activation='relu', input_shape=(14,)),
    ## COPY THE SAME NETWORK HERE
    keras.layers.Dense(units=params_size, activation=None),  # THE OUTPUT IS NOW MADE OF DISTRIBUTION...WE USE HERE A BETA DISTRIBUTION: https://en.wikipedia.org/wiki/Beta_distribution
    tfp.layers.MixtureSameFamily(num_components, tfp.layers.IndependentNormal(event_shape))
])

negloglik = lambda y, p_y: -p_y.log_prob(y)  ## THIS DEFINES THE LOSS - CAN YOU MAKE SENSE OF IT?

pathout='models/mass2'
#define callbacks
from keras.callbacks import TensorBoard
tensorboard = TensorBoard(log_dir=pathout)
callback = tf.keras.callbacks.LearningRateScheduler(lambda e: 0.001 if e < 5 else 0.0001)

model.compile("WHAT SHOULD I PUT HERE?", optimizer='adam', metrics=[])

In [ ]:
history = model.fit(X_train, Y_train, epochs=10, batch_size=100, callbacks=[callback])

In [ ]:
plot(history.history['loss'])

plt.xlabel('Epochs')
plt.ylabel('Loss')

In [ ]:
# Computing mean of the posterior, and output of the previous regression model
# for comparison
Y_pred = scaler.inverse_transform(model(X_test).mean().numpy()).squeeze()
Y_pred_reg = scaler.inverse_transform(regression_model.predict(X_test)).squeeze()

In [ ]:
# Let's plot some posterior distributions
x = linspace(13.51,15.2,100)
# This returns the distribution q(M | x) for all clusters
outputs = model(X_test)
xt = scaler.transform(x.reshape((-1,1)))
logps = []

for i in range(len(x)):
    logps.append(outputs.log_prob(xt[i]).numpy())
logps = np.stack(logps)

for i in range(10):
  figure()
  plot(x, np.exp(logps[:,-i]), label='posterior')
  axvline(Y_true[-i], color='m', label='True value')
  xlabel(r'$\log_{10}[M_{200c}\ (h^{-1}M_\odot)]$')
  legend()

In [ ]:

# PLOT HERE THE PREDICTIONS OF YOUR PROBABILSIITC MODEL ALONG WITH UNCERTAINTIES

### What is my prior?

The output of the Mixture Density Network can be seen as a posterior distribution under the prior defined by the distribution of masses in the training set. 

Let's have a look at this distribution $p(M_{200c})$

In [ ]:
import scipy.stats
hist = np.histogram(scaler.inverse_transform(Y_train), 64)
prior = scipy.stats.rv_histogram(hist)

In [ ]:
plt.hist(scaler.inverse_transform(Y_train), 100, density=True);
x = linspace(13.51,15.2,100)
plot(x, prior.pdf(x))
xlabel(r'$\log_{10}[M_{200c}\ (h^{-1}M_\odot)]$')

So it's definitely not flat, as a matter of fact, it's heavily  preferring lower mass halos. This could explain why our posterior masses are systematically predicted low when we take the mean of the posterior.

Let's compute the mass PDF predicted by the model for  all clusters in the training set.

In [ ]:
# This returns the distribution q(M | x) for all clusters
outputs = model(X_test)
xt = scaler.transform(x.reshape((-1,1)))
logps = []

for i in range(len(x)):
    logps.append(outputs.log_prob(xt[i]).numpy())
logps = np.stack(logps)

Now that we have the prior and posterior distributions, we can replace the training set prior $\tilde{p}$ by a flat prior $p$ simply by using the formula:
$$ q(M_{200c} | x ) \propto \frac{\tilde{p}(M_{200c})}{p(M_{200c})} p(M_{200c} | x) $$

In [ ]:
for i in range(10):
  figure()
  plot(x, np.exp(logps[:,-i]), label='posterior under training prior')
  plot(x, np.exp(logps[:,-i])/prior.pdf(x), label='posterior under flat prior')
  axvline(Y_true[-i], color='m', label='True value')
  xlabel(r'$\log_{10}[M_{200c}\ (h^{-1}M_\odot)]$')
  legend()